# HW07: Parsing

Remember that these homework work as a completion grade. **You can skip one section of this homework.**

In [73]:
from bugs import *

In [74]:
import pandas as pd
import nltk
N = 10000 # # how many datapoints (sentences) to use
df = pd.read_csv('train.zip', names=["label", "title", "lead"])
df["label"] = df["label"].map({1: "world", 2: "sport", 3: "business", 4: "sci/tech"}) 
df["text"] = df["title"] + " " + df["lead"]
df = df.sample(n=N, random_state=0)
df.head()

,label,title,lead,text
40739,sci/tech,First class to the moon,London - British airline magnate Richard Brans...,First class to the moon London - British airli...
105532,sci/tech,Amazon #39;s Holiday Pi,Leave it to Amazon.com (Nasdaq: AMZN). Apparen...,Amazon #39;s Holiday Pi Leave it to Amazon.com...
45004,sci/tech,Will historic flight launch space tourism?,"Regardless, space competitions are poised to b...",Will historic flight launch space tourism? Reg...
71894,world,Thais Drop Peace Bombs On Muslims,(CBS) Millions of folded paper cranes fluttere...,Thais Drop Peace Bombs On Muslims (CBS) Millio...
11970,business,U.S. Economy Grows at Slower Pace Than Expected,Struggling under the weight of a bloated trade...,U.S. Economy Grows at Slower Pace Than Expecte...


In [75]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [76]:
df.text.sample(n=2).apply(nlp)

87539    (Condo, Pandemonium, Nothing, makes, New, York...
69508    (A, Quiet, Struggle, for, the, Conservationist...
Name: text, dtype: object

In [77]:
# preprocess the corpus using spacy or load the pre-processed corpus
df = df.assign(prepro=df.text.apply(nlp))

### Information Extraction

In [78]:
def extract_subject_verb_pairs(sent):
    subjs = [w for w in sent if w.dep_ == "nsubj"]
    pairs = [(w.lemma_.lower(), w.head.lemma_.lower()) for w in subjs]
    return pairs

## extract the subject-verbs pairs and print the result for the first document
sv_pairs = df.prepro.apply(extract_subject_verb_pairs)
first(sv_pairs)

[('class', 'announce'), ('branson', 'announce')]

In [79]:
## create a list ranking the most common pairs and print the first 10 items
pd.Series(Counter(sum(sv_pairs, []))).nlargest(10)

it        be      270
official  say     166
he        be      101
company   say      82
that      be       76
group     say      43
they      be       39
this      be       38
profit    rise     38
police    say      37
dtype: int64

In [80]:
## do the same for verbs-object pairs ('dobj')

def extract(sent):
    x = [w for w in sent if w.dep_ == "dobj"]
    p = [(w.head.lemma_.lower(), w.lemma_.lower()) for w in x]
    return p

pd.Series(Counter(df.prepro.apply(extract).sum())).nlargest(10)

kill      people       76
score     point        44
take      lead         32
win       39;t         28
file      lawsuit      26
reach     agreement    26
cut       job          25
announce  plan         22
hit       homer        22
pay       million      21
dtype: int64

In [81]:
## do the same for adjectives-nouns pairs ('amod')

def extract(sent):
    x = [w for w in sent if w.dep_ == "amod"]
    p = [(w.lemma_.lower(), w.head.lemma_.lower()) for w in x]
    return p

pd.Series(Counter(df.prepro.apply(extract).sum())).nlargest(10)

third         quarter      151
next          year         119
last          week         111
              night         78
open          source        75
first         time          75
chief         executive     69
last          year          67
mobile        phone         65
presidential  election      53
dtype: int64

### Exploring cross label dependencies (Skip)

In [82]:
##TODO extract all the subject-verbs and verbs-object pairs for the verb "win"

In [83]:
##TODO for each label create a list ranking the most common subject-verbs pairs and one for the most common verbs-object pairs
##TODO print the 10 most common pairs for each of the two lists for the labels "sport" and "business"